In [5]:
pip install tensorflow

  Using cached tensorflow-2.7.0-cp37-cp37m-macosx_10_11_x86_64.whl (207.0 MB)
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tensorflow_io_gcs_filesystem-0.22.0-cp37-cp37m-macosx_10_14_x86_64.whl (1.6 MB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached grpcio-1.41.1-cp37-cp37m-macosx_10_10_x86_64.whl (3.9 MB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached libclang-12.0.0-py2.py3-none-macosx_10_9_x86_64.whl (12.2 MB)
  Using cached tensorflow_estimator-2.7.0-py2.py3-none-any.whl (463 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorboard_p

In [43]:
from __future__ import print_function
import seaborn as sns
import numpy as np
import pandas as pd
import collections
from mpl_toolkits.mplot3d import Axes3D
from IPython import display
from matplotlib import pyplot as plt
import sklearn
import sklearn.manifold
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:

# Add some convenience functions to Pandas DataFrame.
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.3f}'.format
def mask(df, key, function):
  """Returns a filtered dataframe, by applying function to key"""
  return df[function(df[key])]

def flatten_cols(df):
  df.columns = [' '.join(col).strip() for col in df.columns.values]
  return df

pd.DataFrame.mask = mask
pd.DataFrame.flatten_cols = flatten_cols

# Install Altair and activate its colab renderer.
print("Installing Altair...")
!pip install git+git://github.com/altair-viz/altair.git
import altair as alt
alt.data_transformers.enable('default', max_rows=None)
alt.renderers.enable('colab')
print("Done installing Altair.")

# Install spreadsheets and import authentication module.
USER_RATINGS = False
!pip install --upgrade -q gspread
from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials

Installing Altair...
  Cloning git://github.com/altair-viz/altair.git to /private/var/folders/hx/6lc74_f56l757kmmkybd34xr0000gn/T/pip-req-build-tyvmgu4g
  Running command git clone -q git://github.com/altair-viz/altair.git /private/var/folders/hx/6lc74_f56l757kmmkybd34xr0000gn/T/pip-req-build-tyvmgu4g
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for altair: filename=altair-4.2.0.dev0-py3-none-any.whl size=812168 sha256=36c37aef47681bdfe84ceeb26ef205fa99714c7193a4ba608b2e7d865491cf32
  Stored in directory: /private/var/folders/hx/6lc74_f56l757kmmkybd34xr0000gn/T/pip-ephem-wheel-cache-8c2bpqnk/wheels/06/13/e0/5bd72c969fe3954ee1561739e5c58e2ddfe5c10fcdffb12faa
Successfully built altair
Done installing Altair.


ModuleNotFoundError: No module named 'google.colab'

## Import data

Import data from our 6 separate files and do some preliminary analysis to better understand what information is contianed in this dataset.

We have 6 files:
- artists
- tags
- user_artists
- user_friends
- user_taggedartists-timestamp
- user_taggedartists


In [5]:
# Load artists
artists_cols = ['id', 'name', 'url', 'pictureURL']
artists = pd.read_csv('Data/artists.dat', sep='	', names=artists_cols, skiprows=1)
artists.head(5)

,id,name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


In [16]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17632 entries, 0 to 17631
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          17632 non-null  int64 
 1   name        17632 non-null  object
 2   url         17632 non-null  object
 3   pictureURL  17188 non-null  object
dtypes: int64(1), object(3)
memory usage: 551.1+ KB


The picture URL contains some null values but we will not need this column for our analysis and se we can drop this column now.

In [20]:
artists.drop('pictureURL', axis=1, inplace=True)
artists.drop('url', axis=1, inplace=True)

In [29]:
artists.head()

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte


This dataframe contains one row for each of the 17,632 artist in this data set with their corresponding id

In [7]:
# Load tags
tags_cols = ['tagID', 'tagValue']
tags = pd.read_csv('Data/tags.dat', sep='	', encoding='latin-1')
tags.head()


,tagID,tagValue
0,1,metal
1,2,alternative metal
2,3,goth rock
3,4,black metal
4,5,death metal


In [30]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11946 entries, 0 to 11945
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tagID     11946 non-null  int64 
 1   tagValue  11946 non-null  object
dtypes: int64(1), object(1)
memory usage: 186.8+ KB


In [33]:
tags.describe()

,tagID
count,11946.000000
mean,6242.315336
std,3667.498057
min,1.000000
25%,3036.250000
50%,6210.500000
75%,9460.750000
max,12648.000000


In [32]:
tags1 = tags[tags['tagValue'].str.endswith('metal')]
tags1.value_counts()

tagID  tagValue             
1      metal                    1
5999   french death metal       1
6318   canadian metal           1
6317   italian metal            1
6311   us metal                 1
                               ..
3135   melodic heavy metal      1
3133   extreme power metal      1
2956   communism death metal    1
2914   transmetal               1
12634  angry metal              1
Length: 306, dtype: int64

This dataframe contains one row for each of the 11,946 seperate tags that can be applied to each artist. As we can see from above there can be a wide variety of different genres. There are 306 different genres that would all be metal. This will need to be a factor when doing analysis.

In [8]:
# Load user-artists
user_artists_cols = ['userID', 'artistID', 'weight']
user_artists = pd.read_csv('Data/user_artists.dat', sep='	')
user_artists.head()

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


In [35]:
user_artists.describe()

,userID,artistID,weight
count,92834.000000,92834.000000,92834.00000
mean,1037.010481,3331.123145,745.24393
std,610.870436,4383.590502,3751.32208
min,2.000000,1.000000,1.00000
25%,502.000000,436.000000,107.00000
50%,1029.000000,1246.000000,260.00000
75%,1568.000000,4350.000000,614.00000
max,2100.000000,18745.000000,352698.00000


In [36]:
user_artists.value_counts()

userID  artistID  weight
2       51        13883     1
1390    964       147       1
        863       687       1
        859       196       1
        709       201       1
                           ..
676     859       168       1
        856       127       1
        854       283       1
        841       198       1
2100    18730     263       1
Length: 92834, dtype: int64

The user artists contains users, the artist they listen to, and the weight which is proportional to how much they have listened to the artist. The weight value goes from 1-352,698 with an average weight of 745. Users may have a weighting for multiple artists.

(README.txt)
92834 user-listened artist relations:
         avg. 49.067 artists most listened by each user
         avg. 5.265 users who listened each artist

In [9]:
# Load user-friends
user_friends_cols = ['userID', 'friendID']
user_friends = pd.read_csv('Data/user_friends.dat', sep='	')
user_friends.head()

,userID,friendID
0,2,275
1,2,428
2,2,515
3,2,761
4,2,831


In [38]:
user_friends.describe()

,userID,friendID
count,25434.000000,25434.000000
mean,992.161437,992.161437
std,603.959049,603.959049
min,2.000000,2.000000
25%,441.000000,441.000000
50%,984.000000,984.000000
75%,1514.000000,1514.000000
max,2100.000000,2100.000000


This dataframe contains: 12717 bi-directional user friend relations, i.e. 25434 (user_i, user_j) pairs
         avg. 13.443 friend relations per user (taken from README.txt with the data)

In [39]:
# Load user-tagged-artists-timestamps
user_tagged_artists_tstamp_cols = ['userID', 'artistID', 'tagID', 'timestamp']
user_tagged_artists_tstamp = pd.read_csv('Data/user_taggedartists-timestamps.dat', sep='	')
user_tagged_artists_tstamp.head()

,userID,artistID,tagID,timestamp
0,2,52,13,1238536800000
1,2,52,15,1238536800000
2,2,52,18,1238536800000
3,2,52,21,1238536800000
4,2,52,41,1238536800000


In [40]:
user_tagged_artists_tstamp.describe()

,userID,artistID,tagID,timestamp
count,186479.000000,186479.000000,186479.000000,1.864790e+05
mean,1035.600137,4375.845328,1439.582913,1.239204e+12
std,622.461272,4897.789595,2775.340279,4.299091e+10
min,2.000000,1.000000,1.000000,-4.287204e+11
25%,488.000000,686.000000,79.000000,1.209593e+12
50%,1021.000000,2203.000000,195.000000,1.243807e+12
75%,1624.000000,6714.000000,887.000000,1.275343e+12
max,2100.000000,18744.000000,12647.000000,1.304941e+12


In [41]:
# Load user-tagged-artists
user_tagged_artists_cols = ['userID', 'artistID', 'tagID', 'day', 'month', 'year']
user_tagged_artists = pd.read_csv('Data/user_taggedartists.dat', sep='	')
user_tagged_artists.head()

,userID,artistID,tagID,day,month,year
0,2,52,13,1,4,2009
1,2,52,15,1,4,2009
2,2,52,18,1,4,2009
3,2,52,21,1,4,2009
4,2,52,41,1,4,2009


In [42]:
user_tagged_artists.describe()

,userID,artistID,tagID,day,month,year
count,186479.000000,186479.000000,186479.000000,186479.000000,186479.000000,186479.000000
mean,1035.600137,4375.845328,1439.582913,1.095566,6.524215,2008.809791
std,622.461272,4897.789595,2775.340279,0.712813,3.486855,1.410062
min,2.000000,1.000000,1.000000,1.000000,1.000000,1956.000000
25%,488.000000,686.000000,79.000000,1.000000,3.000000,2008.000000
50%,1021.000000,2203.000000,195.000000,1.000000,7.000000,2009.000000
75%,1624.000000,6714.000000,887.000000,1.000000,10.000000,2010.000000
max,2100.000000,18744.000000,12647.000000,9.000000,12.000000,2011.000000


The users tagged artist and users tagged artists timestamp are the same data across userID, artistID and tagID columns.

## Data Manipulation and Visualisation

We will bring all of our data together and create some visualisations to better understand the data.


In [44]:
# The indexes in our dataframes start at either 1 or 2 and so we change these ID values to begin at 0 across all our dataframes

#artists
artists["id"] = artists["id"].apply(lambda x: str(x-1))

#tags
tags["tagID"] = tags["tagID"].apply(lambda x: str(x-1))

#user_artists
user_artists["userID"] = user_artists["userID"].apply(lambda x: str(x-2))
user_artists["artistID"] = user_artists["artistID"].apply(lambda x: str(x-1))

#user_friends
user_friends["userID"] = user_friends["userID"].apply(lambda x: str(x-2))
user_friends["friendID"] = user_friends["friendID"].apply(lambda x: str(x-2))

#user tagged artists
user_tagged_artists["userID"] = user_tagged_artists["userID"].apply(lambda x: str(x-2))
user_tagged_artists["artistID"] = user_tagged_artists["artistID"].apply(lambda x: str(x-1))
user_tagged_artists["tagID"] = user_tagged_artists["tagID"].apply(lambda x: str(x-1))

#user tagged artists tstamp
user_tagged_artists_tstamp["userID"] = user_tagged_artists_tstamp["userID"].apply(lambda x: str(x-2))
user_tagged_artists_tstamp["artistID"] = user_tagged_artists_tstamp["artistID"].apply(lambda x: str(x-1))
user_tagged_artists_tstamp["tagID"] = user_tagged_artists_tstamp["tagID"].apply(lambda x: str(x-1))

In [46]:
artists.tail()

,id,name
17627,18740,Diamanda Galás
17628,18741,Aya RL
17629,18742,Coptic Rain
17630,18743,Oz Alchemist
17631,18744,Grzegorz Tomczak


It is noted the index and the id don't add up at the end of the dataframe so we need to be aware of this.

In [47]:
user_artists.tail()

,userID,artistID,weight
92829,2098,18725,337
92830,2098,18726,297
92831,2098,18727,281
92832,2098,18728,280
92833,2098,18729,263


In [48]:
tags.tail()

,tagID,tagValue
11941,12643,suomi
11942,12644,symbiosis
11943,12645,sverige
11944,12646,eire
11945,12647,electro latino


In [49]:
user_tagged_artists.tail()

,userID,artistID,tagID,day,month,year
186474,2098,16436,3,1,7,2010
186475,2098,16436,291,1,5,2010
186476,2098,16436,2086,1,7,2010
186477,2098,16436,2800,1,5,2010
186478,2098,16436,3334,1,7,2010


In [50]:
user_friends.tail()

,userID,friendID
25429,2097,1799
25430,2097,2004
25431,2097,2014
25432,2098,584
25433,2098,605


# Merge then display data


In [ ]:


# Compute the number of movies to which a genre is assigned.
genre_occurences = movies[genre_cols].sum().to_dict()

# Since some movies can belong to more than one genre, we create different
# 'genre' columns as follows:
# - all_genres: all the active genres of the movie.
# - genre: randomly sampled from the active genres.
def mark_genres(movies, genres):
  def get_random_genre(gs):
    active = [genre for genre, g in zip(genres, gs) if g==1]
    if len(active) == 0:
      return 'Other'
    return np.random.choice(active)
  def get_all_genres(gs):
    active = [genre for genre, g in zip(genres, gs) if g==1]
    if len(active) == 0:
      return 'Other'
    return '-'.join(active)
  movies['genre'] = [
      get_random_genre(gs) for gs in zip(*[movies[genre] for genre in genres])]
  movies['all_genres'] = [
      get_all_genres(gs) for gs in zip(*[movies[genre] for genre in genres])]

mark_genres(movies, genre_cols)

# Create one merged DataFrame containing all the movielens data.
movielens = ratings.merge(movies, on='movie_id').merge(users, on='user_id')

# Utility to split the data into training and test sets.
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test